In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from datetime import date
from datetime import datetime
import numpy as np

from autumn.tools.project import get_project
from autumn.settings import Region, Models
from autumn.models.covid_19.constants import AGEGROUP_STRATA, BASE_DATETIME
from autumn.tools.plots.utils import REF_DATE
from autumn.tools.calibration.targets import get_target_series
from autumn.tools import inputs
from autumn.tools.inputs.database import get_input_db
from autumn.tools.utils.pretty import pretty_print
from autumn.tools.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices

In [ ]:
age_integers = [int(group) for group in AGEGROUP_STRATA]
model = Models.COVID_19
region = Region.MYANMAR

## Population

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
project = get_project(model, region)
total_pops = inputs.get_population_by_agegroup(
    AGEGROUP_STRATA, 
    project.param_set.baseline["country"]["iso3"],
    project.param_set.baseline["population"]["region"], 
    year=project.param_set.baseline["population"]["year"]
)
print(f"total modelled ## Mobilitypopulation of {region} is: {round(sum(total_pops) / 1e6, 3)} million")
ax.bar(age_integers, total_pops, width=4)
ax.set_title(region)
ax.set_ylabel("population (millions)")
ax.set_xlabel("starting age of age bracket")
fig.suptitle("population distribution by age")

## Mobility

In [ ]:
input_db = get_input_db()
mob_df = input_db.query("mobility", conditions={"iso3": "MMR"})
mob_df["date_time"] = [datetime.strptime(i, "%Y-%m-%d") for i in mob_df["date"]]
plot_left_date = date(2021, 6, 1)
plot_right_date = mob_df["date_time"].iloc[-1]  # Not sure why this is necessary

mob_fig, mob_axes = plt.subplots(1, 1, figsize=(7, 4))
ax = mob_axes
for mobility_domain in ["grocery_and_pharmacy", "residential", "parks", "retail_and_recreation", "transit_stations"]:
    ax.plot(mob_df["date_time"], mob_df[mobility_domain], label=mobility_domain)
ax.set_ylim((0., 1.4))
ax.tick_params(axis="x", labelrotation=45)
ax.set_title(region)
ax.legend()
mob_fig.suptitle("raw Google mobility domains")
mob_fig.tight_layout()
ax.set_xlim(left=plot_left_date, right=plot_right_date)

Will come back to the modelled location-specific inputs once the new version of this code is merged into master.

## Mixing matrix
### Check how mixing matrix is specified for each region

In [ ]:
print(f"\nFor cluster: {region}")
print(f"\tModelled country: {project.param_set.baseline['country']['iso3']}")
print(f"\tModelled sub-region: {project.param_set.baseline['population']['region']}")
print(f"\tProxy country: {project.param_set.baseline['mixing_matrices']['source_iso3']}")
print(f"\tWhether age adjusted: {project.param_set.baseline['mixing_matrices']['age_adjust']}")

In [ ]:
mixing_matrix = build_synthetic_matrices(
    project.param_set.baseline["country"]["iso3"],
    project.param_set.baseline["mixing_matrices"]["source_iso3"],
    AGEGROUP_STRATA,
    project.param_set.baseline["mixing_matrices"]["age_adjust"],
    project.param_set.baseline["population"]["region"]
)

fig = plt.figure(figsize=(12, 8))
positions = [1, 2, 3, 5, 6]
for i_loc, location in zip(positions, mixing_matrix.keys()):
    ax = fig.add_subplot(2, 3, i_loc)
    ax.imshow(
        np.flipud(np.transpose(mixing_matrix[location])), 
        cmap=cm.hot, 
        vmin=0,
        vmax=mixing_matrix[location].max(), 
        origin="lower"
    )
    ax.set_title(location.replace("_", " "))
    ax.set_xticks([])
    ax.set_yticks([])